In [1]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './examples/tutorials/qm9tut'
if not os.path.exists(qm9tut):
    os.makedirs(qm9tut)

In [2]:
%rm split.npz

qm9data = QM9(
    './examples/tutorials/qm9.db', 
    batch_size=100,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(QM9.U0, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    property_units={QM9.U0: 'eV'},
    num_workers=1,
    split_file=os.path.join(qm9tut, "split.npz"),
    pin_memory=True, # set to false, when not using a GPU
    load_properties=[QM9.U0], #only load U0 property
)
qm9data.prepare_data()
qm9data.setup()

rm: split.npz: No such file or directory


100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


In [3]:
atomrefs = qm9data.train_dataset.atomrefs
print('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')
print('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')
print('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV')

U0 of hyrogen: -13.613121032714844 eV
U0 of carbon: -1029.863037109375 eV
U0 of oxygen: -2042.611083984375 eV


In [4]:
means, stddevs = qm9data.get_stats(
    QM9.U0, divide_by_atoms=True, remove_atomref=True
)
print('Mean atomization energy / atom:', means.item())
print('Std. dev. atomization energy / atom:', stddevs.item())

Mean atomization energy / atom: -4.246610436332558
Std. dev. atomization energy / atom: 0.18493500286437978


In [5]:
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.PaiNN(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)
pred_U0 = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=QM9.U0)
nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_U0],
    postprocessors=[trn.CastTo64(), trn.AddOffsets(QM9.U0, add_mean=True, add_atomrefs=True)]
)

In [6]:
output_U0 = spk.task.ModelOutput(
    name=QM9.U0,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1.,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

In [7]:
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_U0],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [8]:
logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(qm9tut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=qm9tut,
    max_epochs=3, # for testing, we restrict the number of epochs
    accelerator="cpu",
)
trainer.fit(task, datamodule=qm9data)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 39.6 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
39.6 K    Trainable params
0         Non-trainable params
39.6 K    Total params
0.158     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  4.26it/s]

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 100. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 10/10 [00:11<00:00,  0.85it/s, v_num=4, val_loss=7.860]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 10/10 [00:11<00:00,  0.85it/s, v_num=4, val_loss=7.860]


In [9]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(qm9tut, 'best_inference_model'), map_location='cpu')

In [10]:
llpr_model = spk.model.LLPredRigidityNNP(best_model)

In [11]:
weight_dict = {'E': 1, 'F': 1, 'S': 1}
llpr_model.compute_covariance(qm9data.train_dataloader(), weights=weight_dict)

In [12]:
llpr_model.compute_inv_covariance(1e-5, 1e-8)

In [13]:
from schnetpack.utils.llpr import calibrate_llpr_params

calibrate_llpr_params(llpr_model, qm9data.train_dataloader(), energy_key=QM9.U0, function="ssl", n_samples_per_bin=2)

Optimal parameters found beyond the designated parameter space!
Calibrated LLPR parameters:	C = 9.2547E+09	sigma = 6.4591E-07


In [14]:
for batch in qm9data.test_dataloader():
    result = llpr_model(batch)
    print("Result dictionary:", result)
    break

Result dictionary: {'energy_U0': tensor([-12419.0985, -12786.2532,  -9498.9090, -11004.8840, -10935.9251,
        -11511.2783, -11441.8662,  -9923.3172, -11546.4803, -12786.8764,
        -11512.3940, -12141.5461, -11511.3243, -11982.7758, -10499.3846,
        -10568.9323, -11407.3844, -10900.9702, -10569.2442, -11372.0992,
        -11739.6564, -10796.5143, -10970.3892, -12926.1035, -10878.8831,
         -9923.7213, -10569.4217, -12996.2791,  -9452.2065, -11948.4279,
        -11336.8837,  -9831.4842, -11913.3679, -11983.8387, -12489.5510,
        -11546.8487, -12453.9706, -12315.3002, -10832.0862, -10533.8148,
        -10935.4397, -10569.0611, -11878.0266, -10464.7134,  -9935.9791,
        -10604.4055, -11949.2451, -11476.6294, -11947.7303, -12524.5107,
        -11982.7051, -10796.4044, -11948.0333,  -9360.0222,  -9922.5793,
        -11372.3400, -11511.6595, -11948.6420, -11005.9243, -11809.7613,
        -11948.1098, -12210.5134, -11983.3145, -11337.7648, -12524.5614,
        -10533.443

In [17]:
result.keys()

dict_keys(['energy_U0', 'll_feats', 'uncertainty'])

### with forces

In [18]:
import torch
import torchmetrics
import schnetpack as spk
import schnetpack.transform as trn
import pytorch_lightning as pl
import os
import matplotlib.pyplot as plt
import numpy as np

forcetut = './forcetut'
if not os.path.exists(forcetut):
    os.makedirs(forcetut)

In [19]:
from schnetpack.datasets import MD17

ethanol_data = MD17(
    os.path.join(forcetut,'ethanol.db'),
    molecule='ethanol',
    batch_size=10,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(MD17.energy, remove_mean=True, remove_atomrefs=False),
        trn.CastTo32()
    ],
    num_workers=1,
    pin_memory=True, # set to false, when not using a GPU
)
ethanol_data.prepare_data()
ethanol_data.setup()

100%|██████████| 100/100 [00:07<00:00, 12.96it/s]


In [20]:
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)

In [21]:
pred_energy = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=MD17.energy)
pred_forces = spk.atomistic.Forces(energy_key=MD17.energy, force_key=MD17.forces)

In [22]:
nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_energy, pred_forces],
    postprocessors=[
        trn.CastTo64(),
        trn.AddOffsets(MD17.energy, add_mean=True, add_atomrefs=False)
    ]
)

In [23]:
output_energy = spk.task.ModelOutput(
    name=MD17.energy,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.01,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

output_forces = spk.task.ModelOutput(
    name=MD17.forces,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.99,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

In [24]:
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_energy, output_forces],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [25]:
logger = pl.loggers.TensorBoardLogger(save_dir=forcetut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(forcetut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=forcetut,
    max_epochs=3, # for testing, we restrict the number of epochs
    accelerator="cpu",
)
trainer.fit(task, datamodule=ethanol_data)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 2: 100%|██████████| 100/100 [00:10<00:00,  9.39it/s, v_num=2, val_loss=280.0]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 100/100 [00:10<00:00,  9.36it/s, v_num=2, val_loss=280.0]


In [26]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(forcetut, 'best_inference_model'), map_location='cpu')



In [27]:
llpr_model = spk.model.LLPredRigidityNNP(best_model, consider_ll_feat_gradients=True)#, save_ll_feat_per_atom=True)
weight_dict = {'E': 1, 'F': 1, 'S': 1}
# llpr_model.compute_covariance(ethanol_data.train_dataloader(), weights=weight_dict)
# llpr_model.compute_inv_covariance(1e-5, 1e-8)

for batch in ethanol_data.val_dataloader():
    result = llpr_model(batch)
    print("Result dictionary:", result)
    break


Result dictionary: {'energy': tensor([-97210.0654, -97210.7605, -97206.9073, -97209.9240, -97199.7050,
        -97211.4904, -97204.5015, -97205.6022, -97211.4079, -97206.9409],
       dtype=torch.float64, grad_fn=<AddBackward0>), 'forces': tensor([[ 10.6494,  21.3353, -17.2408],
        [ 15.9638,   4.8310, -14.0813],
        [-27.4557,   3.7567,   9.0707],
        [ -6.3261, -13.8017,   5.1199],
        [ -4.5373,  -6.0107,  22.2705],
        [  3.2681, -15.2702,   1.6571],
        [ -9.1851,   2.5905,   2.1877],
        [  3.9397,   4.6313,   6.4225],
        [ 13.6832,  -2.0622, -15.4063],
        [-11.1396,  -9.6098,  12.4538],
        [-20.4713, -26.6090,  -6.5079],
        [ -2.0376,  -6.9788,  -4.8157],
        [ 32.8856,  10.1171, -28.7315],
        [-12.6309,   7.8671,  16.8562],
        [ 13.2541,  -1.9664, -10.1295],
        [  5.5059,  -2.5391,   1.1043],
        [-14.7514,  30.9938,  11.0199],
        [  9.3852,  -1.2750,   8.7502],
        [ -1.0158, -11.8537, -13.1450],


In [29]:
weight_dict = {'E': 1, 'F': 1, 'S': 1}
llpr_model.compute_covariance(ethanol_data.train_dataloader(), weights=weight_dict)

In [30]:
llpr_model.compute_inv_covariance(1e-5, 1e-8)

In [31]:
for batch in ethanol_data.test_dataloader():
    result = llpr_model(batch)
    print("Result dictionary:", result)
    break

Result dictionary: {'energy': tensor([-97204.9312, -97209.5246, -97212.8567, -97206.8458, -97208.1758,
        -97200.7451, -97209.8244, -97206.6482, -97208.6151, -97206.2706],
       dtype=torch.float64, grad_fn=<AddBackward0>), 'forces': tensor([[-19.3452, -24.4908, -33.1036],
        [-15.3924,  34.1064,  -6.9776],
        [-21.6759,  10.8351,  13.2340],
        [  8.7621,   4.4343,  18.5151],
        [  4.6606,  20.3840,   7.1246],
        [ 14.8669,  -2.1212,  -2.3163],
        [ 11.6235, -40.5081,   5.8096],
        [ 10.9932,  -6.5715,  -0.9546],
        [  5.5072,   3.9317,  -1.3313],
        [-10.5130,   5.1577,  14.9365],
        [ -0.1585, -20.9157,  21.9732],
        [ 13.1919,  28.6071,  -2.9437],
        [ -3.0789,  -2.2282,  -3.6631],
        [  5.3025,   1.0228,  -5.1780],
        [ 17.7746,  -6.9433,  -6.3051],
        [ -9.7262,  28.2738, -14.9301],
        [ -1.5465,  -4.9229,  -4.3121],
        [-11.2459, -28.0512,   0.4224],
        [ 13.4215,   6.5385,  -3.6206],


In [1]:
import torch
import torchmetrics
import schnetpack as spk
import schnetpack.transform as trn
import pytorch_lightning as pl
import os
import matplotlib.pyplot as plt

import numpy as np


%rm -r testtut
testtut = './testtut'
if not os.path.exists(testtut):
    os.makedirs(testtut)
    


n_atoms = 4
positions = np.array([
    [5.88321935, 2.88297608, 6.11028356],
    [3.55048572, 4.80964742, 2.77677731],
    [0.40720652, 6.73142071, 3.88666154],
    [2.73860477, 0.96144918, 0.55409947],
])
cell = np.array([
    [6.287023489207423, -0.00034751886075738795, -0.0008093810364881463],
    [0.00048186949720712026, 7.696440684406158, -0.001909478919115524],
    [0.0010077843421425583, -0.0033467698530393886, 6.666654324468158],
])
symbols=["Si"]*n_atoms
energy = np.array([-10169.33552017])
forces = np.array([
    [ 0.02808107, -0.02261815, -0.00868415],
    [-0.03619687, -0.02530285, -0.00912962],
    [-0.03512621,  0.02608594,  0.00913623],
    [ 0.02955523,  0.02289934,  0.0089936 ],
])
stress = np.array([[
    [-2.08967984e-02,  1.52890659e-06,  1.44133597e-06],
    [ 1.52890659e-06, -6.45087059e-03, -7.26463797e-04],
    [ 1.44133597e-06, -7.26463797e-04, -6.04950702e-03],
]])

from ase import Atoms

atoms = []
data = []

for i in range(5):
    atoms.append(Atoms(
        symbols=symbols,
        positions=positions + np.random.rand(4, 3),
        cell=cell,
        pbc=True,
    ))
    
    data.append(dict(
        energy=np.array(energy),
        forces=np.array(forces) + np.random.rand(4, 3),
        stress=np.array(stress) + np.random.rand(3, 3),
    ))


In [2]:
from schnetpack.data import create_dataset, AtomsDataFormat

%rm si.db
db_path = "./si.db"
new_dataset = create_dataset(
    datapath=db_path,
    format=AtomsDataFormat.ASE,
    distance_unit='Ang',
    property_unit_dict=dict(
        energy="eV",
        forces="eV/Ang",
        stress="eV/Ang/Ang/Ang",
    ),
)

new_dataset.add_systems(
    property_list=data,
    atoms_list=atoms,
)

In [3]:
%rm split.npz

custom_data = spk.data.AtomsDataModule(
    './si.db', 
    batch_size=2,
    distance_unit='Ang',
    property_units={'energy':'eV', 'forces':'eV/Ang', 'stress':'eV/Ang/Ang/Ang'},
    num_train=3,
    num_val=2,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        #trn.RemoveOffsets("energy", remove_mean=True, remove_atomrefs=False),
        trn.CastTo32(),
    ],
    num_workers=1,
    pin_memory=True, # set to false, when not using a GPU
)
custom_data.prepare_data()
custom_data.setup()

In [4]:
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
strain = spk.atomistic.Strain()
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)

In [5]:
pred_energy = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key="energy")
pred_forces = spk.atomistic.Forces(energy_key="energy", force_key="forces")#, stress_key="stress", calc_stress=True)

In [6]:
nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],#, strain],
    output_modules=[pred_energy, pred_forces],
)

In [7]:
output_energy = spk.task.ModelOutput(
    name="energy",
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.20,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

output_forces = spk.task.ModelOutput(
    name="forces",
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.40,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

output_stress = spk.task.ModelOutput(
    name="stress",
    loss_fn=torch.nn.MSELoss(),
    loss_weight=0.40,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

In [8]:
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_energy, output_forces],#, output_stress],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [9]:
logger = pl.loggers.TensorBoardLogger(save_dir=testtut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(testtut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=testtut,
    max_epochs=3, # for testing, we restrict the number of epochs
    accelerator="cpu",
)
trainer.fit(task, datamodule=custom_data)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Missing logger folder: ./testtut/lightning_logs

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/sanggyu/miniconda3/envs/schnet/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 2/2 [00:10<00:00,  0.20it/s, v_num=0, val_loss=2.07e+7]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 2/2 [00:10<00:00,  0.20it/s, v_num=0, val_loss=2.07e+7]


In [18]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(testtut, 'best_inference_model'), map_location='cpu')

llpr_model = spk.model.LLPredRigidityNNP(best_model, consider_ll_feat_gradients=True, save_ll_feat_per_atom=True)

for batch in custom_data.train_dataloader():
    print(batch['energy'])
    result = llpr_model(batch)
    print("Result dictionary:", result)
    break


tensor([-10169.3359, -10169.3359])
Result dictionary: {'energy': tensor([3.6537, 3.6408], grad_fn=<SqueezeBackward1>), 'forces': tensor([[ 0.0163, -0.0796, -0.0016],
        [ 0.3519, -0.0390, -0.1039],
        [-0.3601,  0.0364,  0.0942],
        [-0.0082,  0.0822,  0.0113],
        [-0.2754,  0.0590, -0.1150],
        [-0.2872, -0.1061, -0.0509],
        [ 0.2895,  0.1560,  0.0880],
        [ 0.2731, -0.1089,  0.0779]], grad_fn=<NegBackward0>), 'll_feats': tensor([[-1.0263,  1.1874, -0.6239,  0.9357,  2.2836,  2.2531,  0.6957, -1.0439,
         -0.0508,  2.2132,  1.6197,  3.4521, -0.7162, -0.5474,  2.2243],
        [-1.0181,  1.1538, -0.6222,  0.9569,  2.3098,  2.1496,  0.6442, -1.0430,
         -0.0246,  2.0875,  1.6368,  3.4019, -0.6883, -0.5433,  2.0529]],
       grad_fn=<SqueezeBackward1>), 'll_feats_per_atom': tensor([[-0.2566,  0.2990, -0.1553,  0.2315,  0.5668,  0.5754,  0.1814, -0.2608,
         -0.0161,  0.5735,  0.4020,  0.8656, -0.1823, -0.1377,  0.5813],
        [-0.2564,

In [17]:
batch['energy']

tensor([3.6777, 3.6537], grad_fn=<SqueezeBackward1>)

In [20]:
from schnetpack.utils.llpr import calibrate_llpr_params

In [26]:
weight_dict = {'E': 1, 'F': 1, 'S': 1}

llpr_model.compute_covariance(custom_data.train_dataloader(), weights=weight_dict)

calibrate_llpr_params(llpr_model, custom_data.train_dataloader())

Optimal parameters found beyond the designated parameter space!
Calibrated LLPR parameters:	C = 1.7063E+05	sigma = 1.6512E-04
